In [1]:
import numpy as np
import planetary_computer as pc
import stackstac
from pystac_client import Client
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import shapely.geometry 
import rioxarray
from rasterio.crs import CRS
from rasterio.plot import show
import rasterio.features
import matplotlib.colors

In [2]:
aoi = 'fiji'
local =  gpd.read_file(aoi + '.geojson')
area_of_interest = local.geometry[0]
bbox = rasterio.features.bounds(local.geometry[0])
#local.explore()

In [3]:
year = "2020"

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
)

search = catalog.search(
    collections=["alos-fnf-mosaic"], 
    intersects=area_of_interest,
    datetime= year + "-01-01/" + year + "-12-31",
) 
fnf_items = search.get_all_items()
print(f"Returned {len(fnf_items)} items")


count = 0
for item in fnf_items:    
    print(f"{item.id}: {item.datetime}")    
    print(item.assets)
    count = count + 1
print(f"{count} Items")

fnf_item_1 = pc.sign(fnf_items[0])
fnf_item = [pc.sign(item).to_dict() for item in fnf_items]

fnf_tile = stackstac.stack(
    fnf_item, assets=["C"], dtype="uint8", fill_value=0, bounds_latlon=bbox,
).squeeze()

fnf_tile

Returned 0 items
0 Items


IndexError: list index out of range

In [ ]:
classes = fnf_item_1.get_collection().extra_fields["item_assets"]["C"][
    "classification:classes"
]
labels = [x["description"] for x in classes]
labels

In [ ]:
cmap = matplotlib.colors.ListedColormap(
    [
        (0, 0, 0, 0),  # nodata
        (0, 0.7, 0, 1),  # forest, > 90% canopy
        (0.51, 0.94, 0.38, 1),  # forest 10-90% canopy
        (1, 1, 0.6, 1),  # non-forest
        (0.0, 0.0, 1, 1),  # water
    ]
)

# Plot the classification map
#p = fnf_tile[3].plot(cmap=cmap, vmin=0, vmax=4, size=8)

#ticks = np.arange(0.4, 4, 0.8)
#p.colorbar.set_ticks(ticks, labels=labels)
#p.colorbar.set_label("Class")


In [ ]:
#write tiff (fnf_tile)
count = 1
for d in fnf_tile:
    d.rio.write_nodata(0, inplace=True)
    d.rio.to_raster(aoi + "_fnf_" + year + "_" + str(count) + ".tif")
    count = count + 1

In [ ]:
print("Finished.")